In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import numpy as np

In [2]:
datafile_train=r'/Users/61414/Documents/ML/Project1/P1 Data/Consumer_Complaints_train.csv'
datafile_test=r'/Users/61414/Documents/ML/Project1/P1 Data/Consumer_Complaints_test_share.csv'

cd_train=pd.read_csv(datafile_train)
cd_test=pd.read_csv(datafile_test)

In [3]:
cd_test['Consumer disputed?']=np.nan
cd_train['data']='train'
cd_test['data']='test'
cd_all=pd.concat([cd_train,cd_test],axis=0)

In [4]:
cd_all.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,data
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train


In [5]:
cd_all.shape

(598027, 19)

In [6]:
for col in ['Date received','Date sent to company']:
    cd_all[col]=pd.to_datetime(cd_all[col],infer_datetime_format=True)

In [7]:
cd_all['day_diff']=pd.to_numeric(cd_all['Date sent to company']-cd_all['Date received'])
cd_all.dtypes

Date received                   datetime64[ns]
Product                                 object
Sub-product                             object
Issue                                   object
Sub-issue                               object
Consumer complaint narrative            object
Company public response                 object
Company                                 object
State                                   object
ZIP code                                object
Tags                                    object
Consumer consent provided?              object
Submitted via                           object
Date sent to company            datetime64[ns]
Company response to consumer            object
Timely response?                        object
Consumer disputed?                      object
Complaint ID                             int64
data                                    object
day_diff                                 int64
dtype: object

In [8]:
for col in ['Date received','Date sent to company']:
    cd_all.drop([col],1,inplace=True)

In [9]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Sub-product : 47
Issue : 95
Sub-issue : 68
Consumer complaint narrative : 92118
Company public response : 10
Company : 3437
State : 62
ZIP code : 27342
Tags : 3
Consumer consent provided? : 4
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [10]:
cd_all.isnull().sum()

Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Company response to consumer         0
Timely response?                     0
Consumer disputed?              119606
Complaint ID                         0
data                                 0
day_diff                             0
dtype: int64

In [11]:
for col in ['Sub-product','Sub-issue',
            'Company public response','Tags','Consumer consent provided?']:
    varname=col.replace('-','_').replace('?','').replace(" ",'_')+'_isNan'
    cd_all[varname]=np.where(pd.isnull(cd_all[col]),1,0)
    cd_all.drop([col],1,inplace=True)

In [12]:
cd_all.shape

(598027, 18)

In [13]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())

Product : 12
Issue : 95
Consumer complaint narrative : 92118
Company : 3437
State : 62
ZIP code : 27342
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
Consumer disputed? : 2
data : 2


In [14]:
for col in ['ZIP code','Company' ]:
    cd_all.drop([col],1,inplace=True)

cd_all.shape

(598027, 16)

In [15]:
cd_all['Consumer disputed?']=np.where(cd_all['Consumer disputed?']=="Yes",1,0)

In [16]:
k=cd_all['Issue'].value_counts()
for val in k.axes[0][0:10]:
    varname='Issue_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['Issue']==val,1,0)
    
del cd_all['Issue']
cd_all.shape

(598027, 25)

In [17]:
for col in cd_all.select_dtypes(['object']).columns:
    print(col,':',cd_all[col].nunique())
    
cd_all.shape

Product : 12
Consumer complaint narrative : 92118
State : 62
Submitted via : 6
Company response to consumer : 7
Timely response? : 2
data : 2


(598027, 25)

In [18]:
k=cd_all['State'].value_counts()
for val in k.axes[0][0:15]:
    varname='State_'+val.replace(',','_').replace(' ','_')
    cd_all[varname]=np.where(cd_all['State']==val,1,0)
    
del cd_all['State']
cd_all.shape

(598027, 39)

In [19]:
for col in ['Product','Submitted via','Company response to consumer','Timely response?']:
    
    temp=pd.get_dummies(cd_all[col],prefix=col,drop_first=True)
    cd_all=pd.concat([temp,cd_all],1)
    cd_all.drop([col],1,inplace=True)

In [20]:
cd_all.shape

(598027, 58)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer 

In [22]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
lemma = WordNetLemmatizer()
my_stop=set(stopwords.words('english')+list(punctuation))

In [23]:
tfidf= TfidfVectorizer(analyzer='word',min_df=0.01,max_df=0.6,
                       stop_words=my_stop, max_features=200)

In [24]:
cd_all['Consumer complaint narrative'].fillna('missing',inplace=True)

In [25]:
tfidf.fit(cd_all['Consumer complaint narrative'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.6, max_features=200,
                min_df=0.01, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'!', '"', '#', '$', '%', '&', "'", '(', ')', '*',
                            '+', ',', '-', '.', '/', ':', ';', '<', '=', '>',
                            '?', '@', '[', '\\', ']', '^', '_', '`', 'a',
                            'about', ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [26]:
tfidf_data=pd.DataFrame(data=tfidf.transform(cd_all['Consumer complaint narrative']).toarray(),

                       columns=tfidf.get_feature_names())

In [27]:
cd_all.reset_index(drop=True, inplace=True)
tfidf_data.reset_index(drop=True, inplace=True)

In [28]:
cd_combined = pd.concat([cd_all, tfidf_data], axis=1)

In [29]:
cd_combined.shape

(598027, 244)

In [30]:
del cd_combined['Consumer complaint narrative']

In [31]:
cd_combined.shape

(598027, 243)

In [32]:
cd_train=cd_combined[cd_combined['data']=='train']
del cd_train['data']
cd_test=cd_combined[cd_combined['data']=='test']
cd_test.drop(['Consumer disputed?','data'],axis=1,inplace=True)

In [33]:
del cd_combined

In [34]:
cd_train.shape

(478421, 242)

In [35]:
cd_test.shape

(119606, 241)

In [36]:
x=cd_train.drop(['Consumer disputed?','Complaint ID'],1)
y=cd_train['Consumer disputed?']

In [37]:
from sklearn.linear_model import LogisticRegression

In [38]:
clf=LogisticRegression()

In [39]:
clf.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [40]:
prediction=np.where(clf.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [41]:
submission=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction))),
                       columns=['Complaint ID','Consumer disputed?'])

In [49]:
submission.shape

(119606, 2)

In [ ]:
#testing the performance of model on 80:20 trai test split

In [43]:
cd_train1,cd_train2=train_test_split(cd_train,test_size=0.2,random_state=2)

In [44]:
x_train1=cd_train1.drop(['Consumer disputed?','Complaint ID'],1)
y_train1=cd_train1['Consumer disputed?']

In [45]:
x_train2=cd_train2.drop(['Consumer disputed?','Complaint ID'],1)
y_train2=cd_train2['Consumer disputed?']

In [46]:
test_score_logr=clf.predict_proba(x_train2)[:,1]

In [48]:
test_score_logr.shape

(95685,)

In [52]:
from sklearn.metrics import roc_auc_score

In [53]:
# score model performance on the train2 data
roc_auc_score(y_train2,test_score_logr)

0.5

In [ ]:
=========================================================================================

In [ ]:
###grid search

In [54]:
from sklearn.model_selection import GridSearchCV

In [56]:
params={'class_weight':['balanced',None],
        'penalty':['l1','l2'],
        'C':np.linspace(0.01,1000,10)}

In [57]:
grid_search=GridSearchCV(clf,param_grid=params,cv=5,scoring="roc_auc")

In [58]:
grid_search.fit(x_train1,y_train1)

GridSearchCV(cv=5, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': array([1.0000e-02, 1.1112e+02, 2.2223e+02, 3.3334e+02, 4.4445e+02,
       5.5556e+02, 6.6667e+02, 7.7778e+02, 8.8889e+02, 1.0000e+03]),
                         'class_weight': ['balanced', None],
                         'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False

In [59]:
grid_search.best_estimator_

LogisticRegression(C=0.01, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
logr_gs=grid_search.best_estimator_

In [62]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.10f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [63]:
report(grid_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 0.01, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 111.12, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 222.23, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 333.34, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 444.45, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 555.56, 'class_weight': 'balanced', 'penalty': 'l2'}

Model with rank: 1
Mean validation score: 0.5448683557 (std: 0.00254)
Parameters: {'C': 666.67, 'class_weight': 'balanced', 'penalty': 'l2'}

Model wi

In [64]:
test_score_gs=logr_gs.predict_proba(x_train2)[:,1]

In [65]:
roc_auc_score(y_train2,test_score_gs)

0.5287059355830243

In [ ]:
=============================================================================================

In [ ]:
#KFold

In [66]:
from sklearn.model_selection import KFold

In [67]:
alphas=[0.0001,0.0002,0.0005,0.001,0.005,0.01,0.05, 0.1,0.5, 1, 10, 100,500,1000]

auc_10cv =[]
for a in alphas:
    logr=LogisticRegression(C=a,penalty="l2",class_weight="balanced",random_state=2, max_iter=100)
    kf = KFold( n_splits=10)
    auc_for_one_alpha = []
    for train_idx, test_idx in kf.split(x):
        kfold_train_x = x.loc[train_idx]
        kfold_train_y = y.loc[train_idx]
        
        kfold_test_x = x.loc[test_idx]
        kfold_test_y = y.loc[test_idx]

        logr.fit(kfold_train_x, kfold_train_y)

        auc_score_held_out_fold = roc_auc_score(kfold_test_y, logr.predict(kfold_test_x))
        auc_for_one_alpha.append(auc_score_held_out_fold)

    auc_10cv.append(np.mean(auc_for_one_alpha))
best_alpha = alphas[np.argmax(np.array(auc_10cv))]
best_alpha

0.0001

In [68]:
logr_KFold=LogisticRegression(C=best_alpha,penalty="l2",class_weight="balanced",random_state=2)

In [69]:
logr_KFold.fit(x,y)

LogisticRegression(C=0.0001, class_weight='balanced', dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=2, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [70]:
test_score_KFold=logr_KFold.predict_proba(x_train2)[:,1]

In [71]:
roc_auc_score(y_train2,test_score_KFold)

0.5287059355830243

In [ ]:
===========================================================================================

In [ ]:
#Decision tree

In [72]:
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
%matplotlib inline

In [73]:
from sklearn.tree import DecisionTreeClassifier

In [74]:
params={ 'class_weight':[None,'balanced'], 
        'criterion':['entropy','gini'],
        'max_depth':[None,5,10,15,20,30,50,70],
            'min_samples_leaf':[1,2,5,10,15,20], 
            'min_samples_split':[2,5,10,15,20]
       }

In [75]:
clf_dt=DecisionTreeClassifier()

In [92]:
random_search=RandomizedSearchCV(clf_dt,cv=10,
                                 param_distributions=params,
                                 scoring='roc_auc',
                                 n_iter=10
                                    )

In [93]:
random_search.fit(x,y)

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=None,
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    presort='deprecated',
                                                    random_state=None,
         

In [94]:
random_search.cv_results_

{'mean_fit_time': array([17.74732158,  8.62980399, 19.65985053, 14.57136271, 24.15666568,
         5.32770317,  8.55436814, 25.3590842 ,  9.71768196,  5.98770921]),
 'std_fit_time': array([0.89499729, 0.40574582, 1.32720507, 0.85840267, 1.06651403,
        0.3558405 , 0.52068918, 1.76062525, 0.34920207, 0.23209831]),
 'mean_score_time': array([0.0898881 , 0.09011617, 0.09626827, 0.08670948, 0.09471383,
        0.08393345, 0.08351836, 0.10910146, 0.10132394, 0.10057302]),
 'std_score_time': array([0.0079725 , 0.01452529, 0.01351984, 0.0066072 , 0.01217529,
        0.01041538, 0.00361377, 0.01208115, 0.01519211, 0.00655869]),
 'param_min_samples_split': masked_array(data=[10, 15, 5, 15, 5, 10, 15, 2, 5, 20],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[10, 5, 20, 2, 15, 2, 15, 20, 2, 2],
              mask=[False, False, False

In [95]:
random_search.best_estimator_

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [96]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean validation score: 0.6133113758 (std: 0.00247)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 10, 'criterion': 'gini', 'class_weight': None}

Model with rank: 2
Mean validation score: 0.6132351931 (std: 0.00280)
Parameters: {'min_samples_split': 15, 'min_samples_leaf': 15, 'max_depth': 10, 'criterion': 'gini', 'class_weight': None}

Model with rank: 3
Mean validation score: 0.6131892909 (std: 0.00287)
Parameters: {'min_samples_split': 5, 'min_samples_leaf': 2, 'max_depth': 10, 'criterion': 'gini', 'class_weight': 'balanced'}

Model with rank: 4
Mean validation score: 0.6063694108 (std: 0.00282)
Parameters: {'min_samples_split': 20, 'min_samples_leaf': 2, 'max_depth': 5, 'criterion': 'gini', 'class_weight': None}

Model with rank: 5
Mean validation score: 0.6061737920 (std: 0.00271)
Parameters: {'min_samples_split': 10, 'min_samples_leaf': 2, 'max_depth': 5, 'criterion': 'entropy', 'class_weight': None}



In [97]:
dtree=random_search.best_estimator_

In [98]:
dtree.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=10, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=5, min_samples_split=15,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [99]:
test_score_dt=dtree.predict_proba(x_train2)[:,1]

In [100]:
roc_auc_score(y_train2,test_score_dt)

0.6262089685025345

In [101]:
clf_dt.fit(x,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [102]:
prediction_dtree=np.where(clf_dt.predict(cd_test.drop(['Complaint ID'],1))==1,"Yes","No")

In [103]:
submission_dt=pd.DataFrame(list(zip(cd_test['Complaint ID'],list(prediction_dtree))),
                       columns=['Complaint ID','Consumer disputed?'])

In [89]:
import os
os.getcwd()

'C:\\Users\\61414\\Documents\\ML\\Project1'

In [90]:
submission_dt

,Complaint ID,Consumer disputed?
0,675956,No
1,1858795,No
2,32637,No
3,1731374,No
4,501487,No
...,...,...
119601,153482,No
119602,2023523,Yes
119603,1413678,Yes
119604,732458,No


In [104]:
submission_dt.to_csv('sample_submission_run2.csv',index=False)

In [105]:
test_score_dt=dtree.predict_proba(x)[:,1]

In [106]:
roc_auc_score(y,test_score_dt)

0.6266190597818042

In [ ]:
==============================================================================